In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
print("Words: ",words)
print("Classes: ",classes)

Words:  ["'s", ',', 'a', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'fact', 'for', 'fsdh', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'is', 'later', 'lookup', 'me', 'next', 'nice', 'offered', 'provide', 'random', 'see', 'sjdfhk', 'support', 'surprise', 'tell', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'wa', 'what', 'when', 'where', 'who', 'you']
Classes:  ['answer', 'goodbye', 'greeting', 'noanswer', 'options', 'surprise', 'thanks']


In [2]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    print("Sentence words: ",sentence_words)
    return sentence_words

In [3]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    print("Going to clean_up_sentence")
    sentence_words = clean_up_sentence(sentence)
    print("Back to bow")
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    print("Bag: ", np.array(bag))
    return(np.array(bag))

In [4]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    print("Going to bow")
    p = bow(sentence, words,show_details=False)
    print("Back to predict class")
    res = model.predict(np.array([p]))[0]
    print("Res: ",res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    print("Results :", results)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    print("Return list: ",return_list)
    return return_list

In [5]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    print("Result: ",result)
    return result

In [6]:
def chatbot_response(msg):
    print("Going to predict_class")
    ints = predict_class(msg, model)
    print("Ints: ", ints)
    print("Back to chatbot response")
    print("Going to getResponse")
    res = getResponse(ints, intents)
    print("Back to chatbot_response")
    print("Res: ",res)
    if ints[0]['intent'] == 'answer':
        response = res + " " + search(msg)
        return response
    return res

In [7]:
def search(msg):
    return "looking up"

In [10]:
chatbot_response("Who was akbar?")

Going to predict_class
Going to bow
Going to clean_up_sentence
Sentence words:  ['who', 'wa', 'akbar', '?']
Back to bow
Bag:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0]
Back to predict class
Res:  [9.99987006e-01 3.44296822e-08 6.25119938e-08 1.17097716e-05
 1.03634102e-06 1.05302284e-07 1.48935930e-08]
Results : [[0, 0.999987]]
Return list:  [{'intent': 'answer', 'probability': '0.999987'}]
Ints:  [{'intent': 'answer', 'probability': '0.999987'}]
Back to chatbot response
Going to getResponse
Result:  Here's what I found:
Back to chatbot_response
Res:  Here's what I found:


"Here's what I found: looking up"

In [12]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [' ', '', 'fsdh sjdfhk'], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I